<a href="https://colab.research.google.com/github/Jaciss/jai/blob/main/notebooks/bdd_testing_a_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this must happen before %run as they might use these dirs
!mkdir -p features
!mkdir -p features/steps

In [2]:
# import jai.nbpy.bootstrap # that too, as expected; no functions, though. some printing no logging even and no config passing

In [3]:
# c['private_config']='TESTING'
# del c['private_config']
# %run {c['nb_dir']}encrypted_private_config.ipynb

In [4]:
# basic config and bootstrapping all notebooks have
import os.path
from google.colab import drive

nb_name = 'bdd_testing_a_notebook'
try:
  c # try to get config (from remote call)
  c['nb_run_by'] = c.get('nb_name',None) # who is running the nb
  c['nb_name'] = nb_name # nb_name set to local until end
  nb_convert_bootstrap = False
except NameError:
  c = {'nb_name':nb_name, 'nb_run_by':nb_name, 'nb_dir':'drive/MyDrive/Colab\ Notebooks/'} # if local set c
  if not os.path.isdir('/content/drive/'): drive.mount('/content/drive/')
  %run {c['nb_dir']}bootstrap_a_notebook.ipynb
  nb_convert_bootstrap = True 

logger.info(c.get('nb_run_by',None)+'@'+c.get('nb_name',None)) #+' ['+sys.argv[0]+']')

Mounted at /content/drive/


2023-01-08 03:00:05 INFO     bdd_testing_a_notebook@bootstrap_a_notebook [/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py]
2023-01-08 03:00:05 INFO     bdd_testing_a_notebook@bdd_testing_a_notebook


Writing features/bootstrap_a_notebook.feature
bootstrap_a_notebook functions:
No variables match your requested type.


# Functions

In [5]:
# null common magics to run converted notebooks without ipython
# WARNING: this will cause problems if you use magics for "real" functionality
# e.g., things like `%pip install` should be handled via `-r requirements.txt`
# use `replace('get_ipython().system','os.system')` so !commands work
nb_convert_py_prepend = '''
# WARNING: this file is automatically generated, changes will be overwritten
import os
import sys
import logging
logger = logging.getLogger()
if "ipykernel_launcher" not in sys.argv[0]:
  logger.warning("get_ipython() CALLS WILL RETURN '' (empty)")
  class get_ipython():
    @classmethod
    def system(one,two=None,three=None,four=None):
      return os.system(two)
    @classmethod
    def run_cell_magic(one,two=None,three=None,four=None):
      return ''
    @classmethod
    def getoutput(one,two):
      return ''
    @classmethod
    def run_line_magic(one,two,three):
      return ''
'''

In [15]:
def get_module_name(nb_name):
  mod_remove = ['a','notebook','with']
  return ' '.join([word for word in nb_name.split('_') if word not in mod_remove]).replace(' ','')

# notebooks have %%writefile for tests, need to write the py file to avoid module not found
def nb_convert_py(nb_name):
  module_name = get_module_name(nb_name)+'.py'
  command = 'jupyter nbconvert --to python --output-dir jai/nbpy --output '+module_name+' '+c['nb_dir']+nb_name+'.ipynb'
  os.system(command)
  module_name = 'jai/nbpy/'+module_name
  print()
  # insert ipython nullification
  with open(module_name, "r") as fi:
    ti = fi.readlines()
    ti.insert(3, str(nb_convert_py_prepend)) #prepend into test/python file
  with open(module_name, "w") as fo:
    to = ''.join(ti).replace('get_ipython().system','os.system')
    fo.writelines(to.replace('\n\n','\n'))
  return module_name

def test_notebook(nb_to_test:str='.'):
  print('test_notebook()',nb_to_test,'run by',c['nb_run_by'],'self:',sys.argv[0])
  if nb_to_test=='.': module_name = nb_convert_py(c['nb_run_by'])
  else: module_name = nb_convert_py(nb_to_test)
  if c['nb_run_by'] not in sys.argv[0] and "ipykernel_launcher" in sys.argv[0]:
    which_behave = !which behave
    if len(which_behave)==0:
      %pip install behave
      %pip install coverage
    no_upper = 'a|an|and|as|at|but|by|en|for|if|in|of|on|or|the|to|v|via|vs'
    title = ' '.join([x.capitalize() if x not in no_upper.split('|') else x for x in nb_to_test.split('_')]).replace('Bdd','BDD') 
    # title[0].capitalize()
    # !behave
    !coverage run --source=jai.nbpy.{get_module_name(nb_to_test)} -m behave #-q
    !coverage html --title="{title}" #{nb_to_test.replace('_',' ').title()}"
    !coverage json
    return 'Ran tests for '+nb_to_test+' ('+module_name+') use shell command: coverage report'
  return "Running tests on tests causes infinite recursion."

def view_coverage():
  %run {c['nb_dir']}add_notebook_server.ipynb
  #still no way to set media on stock pi/chromium
  with open('htmlcov/style.css','r') as css:
    ugh = css.read()
    ugh = ugh.replace('dark','light')
  with open('htmlcov/style.css','w') as css:
    css.write(ugh)
  %run_local_server

# if c['nb_run_by']!=c['nb_name']:
nb_convert_py(c['nb_name']) #so this file gets included =P
# if nb_convert_bootstrap is True: 
nb_convert_py('bootstrap_a_notebook') #manual import means manual conversion

'jai/nbpy/bootstrap.py'

# Testing

In [7]:
# %run {c['nb_dir']}'encrypted_private_config.ipynb'
# test_notebook('encrypted_private_config')
# %%writefile features/steps/context.py
# import os
# import sys
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '../../')))

# import jai


In [8]:
%%writefile features/{c.get('nb_name')}.feature
Feature: test notebooks using bdd

  Scenario: this notebook is named bdd_testing_a_notebook
    Given we have imported "jai.nbpy.bddtesting"
    When we get configuration key "nb_name"
    Then configuration key "nb_name" is "bdd_testing_a_notebook"

  Scenario: tests calling tests avoids infinite recursion
    Given we have imported "jai.nbpy.bddtesting"
    And we have imported "jai.nbpy.bootstrap"
    When we run test_notebook on "bootstrap_a_notebook"
    Then we see "infinite recursion" in output

Writing features/bdd_testing_a_notebook.feature


In [9]:
%%writefile features/steps/{c.get('nb_name')}.py
from behave import *
import os
from importlib import import_module
import jai


@given(u'we have imported "{nb_name_lower}"')
def step_impl(context, nb_name_lower):
  try:
    len(context.loaded_modules)
  except:
    context.loaded_modules = {}
  
  print('importing '+nb_name_lower)
  context.loaded_modules[nb_name_lower] = import_module(nb_name_lower)
  context.c=context.loaded_modules[nb_name_lower].c
  print('loaded modules',context.loaded_modules)
  print('context.c is',context.c)

@given(u'we have imported the notebook for this step')
def step_impl(context):
  module_name = os.path.basename(__file__).replace('.py','') #this uses the *step* notebook name
  print('module_name:',module_name)
  imported_module = import_module(module_name)
  context.c = imported_module.c

@when(u'we get configuration key "{key_name}"')
def step_impl(context, key_name):
  val = context.c.get(key_name,None)
  assert val is not None

@then(u'configuration key "{key_name}" is "{val}"')
def step_impl(context, key_name, val):
  assert context.c[key_name]==val

# @then(u'we will see "{to_match}" in the output')
# def step_impl(context, to_match):
#   assert to_match in context.output == True

@then(u'we see "{to_match}" in output')
def step_impl(context, to_match):
  #os.system('coverage')
  # output = os.popen('coverage report').read()
  # print('coverage report:',output)
  assert to_match in context.output

@then(u'"{var1}" equals "{var2}"')
def step_impl(context, var1, var2):
  print('c3 is',context.c)
  assert context.c[var1]==var2
  assert context.failed is False

@when(u'we run test_notebook on "{function_arg}"')
def step_impl(context, function_arg):
    import sys
    #print('modules',sys.modules.keys())
    from jai.nbpy.bddtesting import test_notebook
    print('jainbpy!',test_notebook)
    #context.output = context.loaded_modules['jai.nbpy.bddtesting'].test_notebook(function_arg) 
    context.output = test_notebook(function_arg)
    print('context.output:',context.output)

Writing features/steps/bdd_testing_a_notebook.py


In [16]:
if c['nb_name'] == c['nb_run_by'] and "ipykernel_launcher" in sys.argv[0]: 
  test_notebook(c['nb_name'])
  view_coverage()

test_notebook() bdd_testing_a_notebook run by bdd_testing_a_notebook self: /usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py

Feature: test notebooks using bdd # features/bdd_testing_a_notebook.feature:1

  Scenario: this notebook is named bdd_testing_a_notebook        # features/bdd_testing_a_notebook.feature:3
    Given we have imported "jai.nbpy.bddtesting"                 # features/steps/bdd_testing_a_notebook.py:7
[NbConvertApp] Converting notebook drive/MyDrive/Colab Notebooks/bdd_testing_a_notebook.ipynb to python
[NbConvertApp] Writing 8200 bytes to jai/nbpy/bddtesting.py
[NbConvertApp] Converting notebook drive/MyDrive/Colab Notebooks/bootstrap_a_notebook.ipynb to python
[NbConvertApp] Writing 3269 bytes to jai/nbpy/bootstrap.py
    Given we have imported "jai.nbpy.bddtesting"                 # features/steps/bdd_testing_a_notebook.py:7 2.745s
    When we get configuration key "nb_name"                      # features/steps/bdd_testing_a_notebook.py:27
    When we 

<IPython.core.display.Javascript object>

Now serving Jai at: https://l7b75bszrmi-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [11]:
print(c['nb_name'])

bdd_testing_a_notebook


# EOF

In [12]:
c['nb_name'] = c['nb_run_by'] # return name to calling script, if any

In [13]:
print(c['nb_name'])

bdd_testing_a_notebook
